In [15]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Audi-Stock-Price-Prediction'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [16]:
data = pd.read_csv('./data/audi.csv')

In [17]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-08-24,701.549988,701.549988,701.549988,701.549988,699.382935,0
1,2016-08-25,701.549988,701.549988,701.549988,701.549988,699.382935,0
2,2016-08-26,701.549988,701.549988,701.549988,701.549988,699.382935,0
3,2016-08-29,701.549988,701.549988,701.549988,701.549988,699.382935,0
4,2016-08-30,698.049988,710.000000,695.700012,695.700012,693.551025,300


In [18]:
data = data['Close']

In [19]:
data = data.tolist()

In [20]:
data = torch.from_numpy(np.array(data))

In [21]:
data = data.view(1,-1)

In [22]:
data_input = data[:3,:-1].float().to(device)
data_target = data[:3,1:].float().to(device)

In [27]:
class Model(Module):
    def __init__(self,hidden=128):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t1 = h_t1.to(device)
        c_t1 = c_t1.to(device)
        h_t2 = h_t2.to(device)
        c_t2 = c_t2.to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t1)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t1)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)

In [28]:
model = Model()

In [29]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [30]:
criterion = MSELoss()

In [31]:
epochs = 100

In [32]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in range(epochs):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        preds = preds.to(device)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        preds = preds.to(device)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds.cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        n = data_input.shape[1]
        plt.plot(np.arange(n),data_target.cpu().view(-1).detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: Input batch size 1 doesn't match hidden[0] batch size 128